In [11]:
using HDF5
using LinearAlgebra

# File name (should match the Python script)
file_name = "QbData.h5"

function read_examples_data(file_name)
    # Open the HDF5 file and read datasets
    h5open(file_name, "r") do file
        Y1 = read(file["Y1"])
        Y2 = read(file["Y2"])
        Uy = read(file["Uy"])
        return Y1, Y2, Uy
    end
end 

read_examples_data (generic function with 1 method)

In [5]:
Y1, Y2, Uy = read_examples_data(file_name)

([0.0 0.0 -1.0; -0.0067893268258513925 0.12081174211125437 -0.9873688632008858; … ; 0.2958830041967634 0.4496502209469271 -0.8180174086911732; 0.1453314394896256 0.4485483872330605 -0.8680783573065066], [-0.0067893268258513925 0.12081174211125437 -0.9873688632008858; -0.09750737214167098 0.19772837038745095 -0.9760786389581715; … ; 0.1453314394896256 0.4485483872330605 -0.8680783573065066; -0.03513761568681435 0.36508952230979097 -0.9024177854511133], [1.0; 0.9385563838770445; … ; -0.566451455811455; -0.8615998483849043;;])

In [7]:
size(Y1)

(790, 3)

In [6]:
# Helper function: Column-wise Kronecker product (Khatri-Rao product)

function khatri_rao(U, X)

    if ndims(U) == 1
        U = reshape(U, 1, :)
    end

    # Input: U ∈ R^(Nc×M), X ∈ R^(n×M)
    # Output: U ⊙ X ∈ R^(n*Nc×M)
    Nc, M = size(U)
    n, Mx = size(X)
    if M != Mx
        throw(DimensionMismatch("Number of columns in U and X must match"))
    end

    result = zeros(n * Nc, M)  # Preallocate output matrix
    for j in 1:M
        result[:, j] = vec(kron(U[:, j], X[:, j]))
    end
    return result
end


# Algorithm 1: Bilinear Dynamic Mode Decomposition (biDMD)

"""
Main function: Performs Bilinear Dynamic Mode Decomposition (biDMD) to extract intrinsic dynamics (A) and control dynamics (B).

Inputs:
- X: Snapshot matrix of state data
- X_prime: Shifted snapshot matrix of state data
- U: Control input matrix
- r_tilde: Target rank for SVD on combined matrix Ξ
- r_hat: Target rank for SVD on shifted snapshot matrix X_prime

Outputs:
- Φ: DMD modes
- Λ: DMD eigenvalues
- A_tilde: Intrinsic dynamics matrix (low-rank approximation)
- B_tilde: Control dynamics matrix
"""

function biDMD(X, X_prime, U, r_tilde, r_hat)
    # Step 1: Debug dimensions
    println("Dimensions of X: ", size(X))
    println("Dimensions of X_prime: ", size(X_prime))
    println("Dimensions of U: ", size(U))

    # Compute Khatri-Rao product
    U_kron_X = khatri_rao(U, X)
    println("Dimensions of U_kron_X (Khatri-Rao): ", size(U_kron_X))

    # Step 2: Construct combined matrix Ξ
    Ξ = vcat(X, U_kron_X)  # Ensure dimensions align
    println("Dimensions of Ξ: ", size(Ξ))

    # Step 3: Perform truncated SVD on Ξ to rank r_tilde
    U_tilde, Σ_tilde, V_tilde = svd(Ξ)
    U_tilde = U_tilde[:, 1:r_tilde]
    Σ_tilde = Σ_tilde[1:r_tilde]
    V_tilde = V_tilde[:, 1:r_tilde]

    # Decompose U_tilde into U_tilde_A and U_tilde_B
    n = size(X, 1)
    U_tilde_A = U_tilde[1:n, :]
    U_tilde_B = U_tilde[n+1:end, :]

    # Step 4: Compute estimates for A and B
    A_tilde = X_prime * V_tilde * inv(Diagonal(Σ_tilde)) * U_tilde_A'
    B_tilde = X_prime * V_tilde * inv(Diagonal(Σ_tilde)) * U_tilde_B'

    # Step 5: Perform truncated SVD on X_prime to rank r_hat
    U_hat, Σ_hat, V_hat = svd(X_prime)
    U_hat = U_hat[:, 1:r_hat]
    Σ_hat = Σ_hat[1:r_hat]
    V_hat = V_hat[:, 1:r_hat]

    # Step 6: Low-rank approximation for A
    A_hat = U_hat' * A_tilde * U_hat

    # Step 7: Eigendecomposition of A_hat
    W, Λ = eigen(A_hat)

    # Step 8: Compute DMD modes Φ
    Φ = A_tilde * U_hat * W

    return Φ, Λ, A_tilde, B_tilde
end


biDMD (generic function with 1 method)

In [12]:
Φ, Λ, A_tilde, B_tilde = biDMD(Y2', Y1', Uy', 3, 3)

Dimensions of X: (3, 790)
Dimensions of X_prime: (3, 790)
Dimensions of U: (1, 790)
Dimensions of U_kron_X (Khatri-Rao): (3, 790)
Dimensions of Ξ: (6, 790)


(ComplexF64[0.2260826966930716 + 0.5235556654539786im, 1.1140845644514379 - 0.006157242186946766im, 1.1410066330119453 - 0.09843345678821055im], ComplexF64[-0.002464731634310611 + 0.6938898562072803im -0.002464731634310611 - 0.6938898562072803im 0.20143556012543387 + 0.0im; -0.6968248486373606 - 0.0im -0.6968248486373606 + 0.0im 0.03623139387099492 + 0.0im; -0.031313735598331185 - 0.17878862613434743im -0.031313735598331185 + 0.17878862613434743im 0.9788314467849495 + 0.0im], [0.920188337831139 0.38610613958711965 -0.006484415695506784; -0.3684139594937002 0.9076421380280718 0.02356172106639015; -0.011400020857109366 0.0053050073855349655 0.9767678812967444], [0.010956532284682123 0.00391188764151054 0.059053862259440336; -0.004106102392455591 0.015274878765980918 -0.07302490773301522; 0.09651837571171851 -0.06372312953194337 0.1283000427935174])

In [15]:
A_tilde

3×3 Matrix{Float64}:
  0.920188  0.386106    -0.00648442
 -0.368414  0.907642     0.0235617
 -0.0114    0.00530501   0.976768

In [16]:
B_tilde

3×3 Matrix{Float64}:
  0.0109565   0.00391189   0.0590539
 -0.0041061   0.0152749   -0.0730249
  0.0965184  -0.0637231    0.1283